In [1]:
using Oceananigans
using Printf
using GLMakie

using Random, Statistics

In [2]:
function run_convective_instability(Δb)
    L = 10
    H = L
    grid = RectilinearGrid(size=(256, 256), x=(-L/2, L/2), z=(-H/2, H/2),
                           topology=(Periodic, Flat, Bounded))
    
    b₀(x, z, t, p) = (p.Δb/2)*tanh(z / p.h)
    
    B = BackgroundField(b₀, parameters=(h=1/4,Δb=Δb));
    
    zF = znodes(grid, Face())
    zC = znodes(grid, Center())
    
    h, Δb = B.parameters
    
    # The model
    model = NonhydrostaticModel(timestepper = :RungeKutta3,
                                  advection = UpwindBiased(order=5),
                                       grid = grid,
                                   coriolis = nothing,
                          background_fields = (b=B,),
                                    closure = ScalarDiffusivity(ν=0.5e-4, κ=0.5e-4),
                                   buoyancy = BuoyancyTracer(),
                                    tracers = :b)
    
    Δt = 0.005
    simulation = Simulation(model, Δt=Δt, stop_iteration=8000, verbose=false)
    
    u, v, w = model.velocities
    b = model.tracers.b
    xb, yb, zb = nodes(b)
    total_b = Field(b + model.background_fields.tracers.b)
    
    mean_perturbation_kinetic_energy = Field(Average(1/2 * (u^2 + w^2)))
    
    noise(x, z) = 1.e-3*randn()
    set!(model, u=noise, w=noise, b=noise)
    
    
    filename = Δb <= 0 ? "unstable" : "stable"
    simulation.output_writers[:buoyancy] =
        JLD2Writer(model, (b=b, B=total_b),
                         schedule = TimeInterval(0.2),
                         filename = string("../data/raw_simulation_output/rayleigh_taylor_instability/",filename,".jld2"),
                         overwrite_existing = true)
    
    @info "*** Running a simulation of Rayleigh-Taylor instability..."
    run!(simulation)

    @info string("Making a neat movie of convectively ",filename," flow...")

    filepath = simulation.output_writers[:buoyancy].filepath
    
    B_timeseries = FieldTimeSeries(filepath, "B")
    
    times = B_timeseries.times
    t_final = times[end]
    
    n = Observable(1)
    
    Bₙ = @lift interior(B_timeseries, :, 1, :, $n)
    
    fig = Figure(size=(550, 550))
    
    kwargs = (xlabel="x [m]", ylabel="z [m]", limits = ((-L/2, L/2), (-H/2, H/2)), aspect=1,)
    
    title = @lift @sprintf("buoyancy [m/s²] at t = %.2f seconds", times[$n])
    
    ax_B = Axis(fig[1, 1]; title = title, kwargs...)
    
    B_lims = (-1., 1.)
    
    hm_B = heatmap!(ax_B, xb, zb, Bₙ; colorrange = B_lims, colormap = :balance)
    Colorbar(fig[1, 2], hm_B)
    
    frames = 1:length(times)
    
    record(fig, string("../movies/Lecture19_rayleigh_taylor_",filename,".mp4"), frames, framerate=8) do i
        n[] = i
    end

end

run_convective_instability (generic function with 1 method)

In [3]:
run_convective_instability(2)
run_convective_instability(-2)

[ Info: *** Running a simulation of Rayleigh-Taylor instability...
[ Info: Making a neat movie of convectively stable flow...
[ Info: *** Running a simulation of Rayleigh-Taylor instability...
[ Info: Making a neat movie of convectively unstable flow...


"../movies/Lecture19_rayleigh_taylor_unstable.mp4"